# Auxiliary.ipynb

#### Notebook containing computations that multiple data analysis notebooks need, such as waveform reading, surrogate evaluation, overlap computation, and SNR computation

Maria Okounkova (mokounkova@flatironinstitute.org)

In [13]:
import matplotlib.pyplot as plt
import h5py
from astropy import constants as const
import seaborn as sns
import numpy as np
from math import pi
import matplotlib
from scipy.interpolate import InterpolatedUnivariateSpline
import gwsurrogate
from pycbc.detector import Detector
import pycbc
from pycbc.filter.matchedfilter import overlap

### General auxiliary methods - subtracting peak times, computing $\Delta t$, ramp function, etc

In [8]:
def GetPeakTime(time, data): 
    """ Grab the peak time of some data """
    t_peak = time[np.argmax(data)]
    return t_peak

def SubtractPeakTime(time, data): 
    """ Subtract the peak time of some data """
    t_peak = GetPeakTime(time, data)
    return time - t_peak

def dt_eval(time):
    """ Return the time step of a given time array """
    return (time[1] - time[0])

def df_eval(time):
    """ Return the delta_f of a given time array """
    delta_t = dt_eval(time)
    return 1.0/((time[-1] - time[0]) + delta_t)

def Ramp(time, t_s, t_r):
    """ Ramp function for tapering the waveform"""
    if (time < t_s):
        return 0.0
    elif time > (t_s + t_r):
        return 1.0
    else:
        t = (time - t_s)/t_r
        return t**5*(126 + t*(-420 + t*(540 + t*(-315 + 70*t))))

### dCS auxiliary methods - converting dimensionless coupling constant to kms, etc

In [2]:
def EllinKm(ell, mass):
    """ Return the value of the dCS coupling constant in km """
    if 'p' in ell:
        ## If we're using a string like 0p0 for 0.0, convert to a float
        ell = float(ell.replace('p', '.'))
    mass_msun = mass * const.M_sun
    phys_ell_km = ell * mass_msun * const.G /(const.c**2) / 1000
    return phys_ell_km.value

### NR auxiliary methods - waveform reading methods

In [9]:
def swsh(s, modes, theta, phi, psi=0):
    """
    Return a value of a spin-weighted spherical harmonic of spin-weight s. 
    If passed a list of several modes, then a numpy array is returned with 
    SWSH values of each mode for the given point.
    For one mode:       swsh(s,[(l,m)],theta,phi,psi=0)
    For several modes:  swsh(s,[(l1,m1),(l2,m2),(l3,m3),...],theta,phi,psi=0)
    """
    import spherical_functions as sf
    import quaternion as qt
    return sf.SWSH(qt.from_spherical_coords(theta, phi), s, modes) * np.exp(1j * s * psi)

In [10]:
def ReadExtrapolatedModes(file, params_dict, interpolate = True):
    """ 
        File is the file containing the extrapolated waveform that we want to read in.
        For params_dict, 
        mass_msun is the total mass of the system in solar masses, and 
        dist_mpc is the distance to the system in kpc. 
        theta and phi are angles determining the inclination.
        dt is the timestep (reciprocal of the sampling rate)
        
        If we want to interpolate the waveform to have even timesteps dt, then 
        set interpolate to True. Otherwise, we'll return the data without 
        performing the interpolation 

        """

    ## Convert distance to kpc and mass into solar masses
    mass = params_dict['mass']
    dist_mpc = params_dict['dist_mpc']
    theta = params_dict['theta']
    phi = params_dict['phi'] 
    dt = params_dict['dt']
    dist_kpc = dist_mpc * 1000 * const.kpc
    mass_msun = mass * const.M_sun
    
    ## Read in the data
    f = h5py.File(file, 'r')
    
    ## grab the length of the waveform first
    data = f['Extrapolated_N2.dir']['Y_l2_m2.dat']
    time = np.array(data[:,0])
    
    h_plus = np.zeros(len(time))
    h_cross = np.zeros(len(time))

    modes = [(l,m) for l in range(2,5) for m in range(-l, l+1)]
    for mode in modes: 
        
        ## Grab the mode in question
        l = mode[0]
        m = mode[1]
        
        data = f['Extrapolated_N2.dir']['Y_l' + str(l) + '_m' + str(m) + '.dat']
        real = np.array(data[:,1])
        imag = np.array(data[:,2])
        coeff = real + 1j * imag
        
        ## Multiply by the corresponding spin-weighted spherical harmonic
        Ylm = swsh(-2, [(l,m)], theta=theta, phi=phi, psi=0) 
        h = coeff * Ylm 
        
        ## Add to our h_plus and h_cross computations
        h_plus = h_plus + np.real(h)
        h_cross = h_cross - np.imag(h) 
        
    ## Apply the astrophysical parameters
    time = time*mass_msun*const.G/(const.c**3)
    h_plus = h_plus*const.G*mass_msun/((const.c)**2*dist_kpc)
    h_cross = h_cross*const.G*mass_msun/((const.c)**2*dist_kpc)

    ## Taper the waveform and apply the ramp (need to start the waveform at zero for this)
    time = time - time[0]
    ramp = np.array([Ramp(t.value, 0.1, 0.3) for t in time])
    
    h_plus = h_plus * ramp
    h_cross = h_cross * ramp
    
    ## Now subtract off the peak time (this makes the spine interpolation easier)
    amp = np.sqrt(h_plus**2 + h_cross**2)
    time = time - time[np.argmax(amp)]
    
    if not interpolate:
        print("Not performing the interpolation")
        return time, h_plus, h_cross, np.sqrt(h_plus**2 + h_cross**2)
    
    ## Now build the interpolants 
    cs_plus = InterpolatedUnivariateSpline(time, h_plus)
    cs_cross = InterpolatedUnivariateSpline(time, h_cross)

    ## Now create an evenly-spaced time array and interpolate the data 
    time_cs = np.arange(time[0].value, time[-1].value, dt)

    h_plus_cs = cs_plus(time_cs) 
    h_cross_cs = cs_cross(time_cs) 
    
    ## Return these new interpolated values
    return time_cs, h_plus_cs, h_cross_cs, np.sqrt(h_plus_cs**2 + h_cross_cs**2)

### Surrogate auxiliary methods - surrogate model evaluation

In [16]:
def EvaluateSurrogate(sur, params_dict):
    """ Evaluate the surrogate waveform using a dictionary of parameters
        and an instance of a surrogate model """

    data = sur(params_dict['q'], params_dict['a_1'], params_dict['a_2'], \
               dt = params_dict['dt'], units = 'mks', M = params_dict['mass'], \
               dist_mpc = params_dict['dist_mpc'], f_low = params_dict['f_low'], \
               inclination = params_dict['theta'], ellMax = 4, \
               phi_ref = params_dict['phi'])

    time = np.array(data[0])
    h_plus = np.real(data[1])
    h_cross = -1 * np.imag(data[1])

    ## Taper the waveform and apply the ramp (need to start the waveform at zero for this)
    #time = time - time[0]
    #ramp = np.array([Ramp(t, 0.1, 0.3) for t in time])
    
    #h_plus = h_plus * ramp
    #h_cross = h_cross * ramp
    
    ## Subtract off the peak time of the waveform
    amp = np.sqrt(h_plus**2 + h_cross**2)
    time = time - time[np.argmax(amp)]
    
    return time, h_plus, h_cross, np.sqrt(h_plus**2 + h_cross**2)

### Detector auxiliary methods - $h_\times$ and $h_\times$ to detectors, padding time segments

In [11]:
def ProjectToDetectors(ra, dec, pol, t0, plus, cross, time):
    """
    Given plus and cross gravitational waveforms as a function of time, 
    project to H1 and L1 using the angles: 
    ra - Right Ascension
    dec - Declination
    pol - Polarization Angle
    t0 - reference time for when signal reaches Hanford
    """
    
    ## Grab the dt
    dt = dt_eval(time)
    
    d_H1 = Detector("H1")
    d_L1 = Detector("L1")
    
    # The time delay of the signal between the detectors
    t_delay_LH = d_L1.time_delay_from_detector(d_H1, ra, dec, t0)
    
    # Round the delay time to the nearest dt (this assumes that t_gps is a multiple of dt)
    t_delay_rounded_LH = round(t_delay_LH / dt) * dt

    #Antenna Patterns
    Fp_H1, Fc_H1 = d_H1.antenna_pattern(ra, dec, pol, t0)
    Fp_L1, Fc_L1 = d_L1.antenna_pattern(ra, dec, pol, t0 + t_delay_rounded_LH)
    
    #project
    h_H1 = Fp_H1*plus + Fc_H1*cross
    h_L1 = Fp_L1*plus + Fc_L1*cross
    
    #Shift times
    time_H1 = time
    time_L1 = time + t_delay_rounded_LH
    
    return time_H1, h_H1, time_L1, h_L1

In [10]:
def PadZeroes(time, strain, t_gps, peak_time_in_segment, segment_length):
    """ 
    Given strain data as a function of time, pad the data with zeros in order
    to obtain a desired segment_length, where
    
    t_gps: gps peak time of the signal
    peak_time_in_segmnet: how far into the segment we want the peak of the waveform to be
    segment_length: the total length of the segment 
    """

    ## Pad the data with zeros -- we want to pad both the H1 and L1 data so that 
    ## they span the same gps times
    dt = dt_eval(time)
    segment_start_time = t_gps - peak_time_in_segment
    segment_end_time = segment_start_time + segment_length
    
    ## Do Hanford first (see how much we need to pad in order to get )
    start_pad_time = time[0] - segment_start_time
    end_pad_time = segment_end_time - time[-1]
    
    ## Figure out how many integer zeroes to pad the data
    start_pad_zeros = start_pad_time / dt
    end_pad_zeros = end_pad_time / dt
    
    ## Pad the data with zeroes 
    strain_padded = np.pad(strain, (int(start_pad_zeros), int(end_pad_zeros)), 'constant', constant_values=(0.0, 0.0))
    time_padded = np.arange(0., segment_length + dt, dt) + segment_start_time
    if (len(time_padded) > len(strain_padded)):
        time_padded = np.arange(0., segment_length, dt) + segment_start_time

    return time_padded, strain_padded

In [9]:
def PadAndProject(time, h_plus, h_cross, params_dict):
    """ Given a time, h_plus, and h_cross array, pad the data to the desired segment length, and project 
        to detectors.
        Within params_dict, 
        t_gps is the gps time of the event
        peak_time_in_segment is the time within the segment corresponding to the peak of the waveform (in sec)
        segmentt_length is the length of the segment (in sec)
        ra is the right ascention
        dec is the declination
        pol is the polarization
    """
       
    ## Grab the dt
    dt = dt_eval(time)
    
    ## The present peak time in the gravitational waveform (in secs)
    ## This is not the gps time, but rather should be ~0 from the surrogate / NR waveform 
    t_peak = time[np.argmax(h_plus**2 + h_cross**2)]
    
    ## Shift the times by the gps time, so that the peak of the waveform (in Handford)
    ## is now at t_gps
    time = time + params_dict['t_gps']

    ## Double check the peak time
    t_peak = time[np.argmax(h_plus**2 + h_cross**2)]
    
    ## Project data to the detectors -- both timeH and timeL % dt will be zero
    timeH, strainH, timeL, strainL = ProjectToDetectors(ra=params_dict['ra'], dec=params_dict['dec'], \
                                                        pol=params_dict['pol'], t0=params_dict['t_gps'], \
                                                        plus=h_plus, cross=h_cross, time=time)
    
    ## Pad the data with zeros
    timeH, strainH = PadZeroes(timeH, strainH, params_dict['t_gps'], params_dict['peak_time_in_segment'], \
                                               params_dict['segment_length'])
    timeL, strainL = PadZeroes(timeL, strainL, params_dict['t_gps'], params_dict['peak_time_in_segment'], \
                                               params_dict['segment_length'])
    
    ## Return the projected data
    return timeH, strainH, timeL, strainL

### SNR and Overlap Auxiliary methods - Computing single and multi detector overlaps, computing SNRs

In [12]:
def ComputeInnerProduct(time1, strain1, time2, strain2, normalized = False):
    """ Given two time-domain strains and the corresponding time arrays, compute the 
        inner product in one detector using pycbc. 
        Use normalized = True for overlaps, and normalized = False for SNR
        Assuming that this is Eq. 4 in https://arxiv.org/abs/2003.09456
        If we want it normalized, then it's like Eq. 8 in https://arxiv.org/abs/2003.09456
    """

    ## Construct time array to be interpolated onto, since the pycbc overlap computation
    ## requires the time arrays to be the same
    delta_t = dt_eval(time1)
    time = np.arange(start = max(time1[0], time2[0]), stop = min(time1[-1], time2[-1]), step = delta_t)
    delta_f = df_eval(time)
    
    ## Interpolate strains onto time array
    cs1 = InterpolatedUnivariateSpline(time1, strain1)
    cs2 = InterpolatedUnivariateSpline(time2, strain2)
    
    strain1 = cs1(time) 
    strain2 = cs2(time)

    ## Read in PSD and construct interpolant
    psd_file = "PSDs/design/aLIGOZeroDetHighPower-PSD.txt"
    psd_frequencies, psd_vals = np.loadtxt(psd_file, comments="#",usecols=([0,1]),unpack=True)
    cs = InterpolatedUnivariateSpline(psd_frequencies, psd_vals)

    ## Interpolated PSD onto df-spaced values
    freqs = delta_f * np.array(range(len(time)))
    psd = cs(freqs)

    ## Timeseries and Frequency series objects for pycbc computation
    strain1_ts = pycbc.types.timeseries.TimeSeries(strain1, delta_t = delta_t, epoch = time[0])
    strain2_ts = pycbc.types.timeseries.TimeSeries(strain2, delta_t = delta_t, epoch = time[0])
    psd_fs = pycbc.types.frequencyseries.FrequencySeries(psd, delta_f = delta_f, epoch = time[0])

    overlap_val = overlap(vec1 = strain1_ts, vec2 = strain2_ts, psd = psd_fs, normalized = normalized, \
                          low_frequency_cutoff = 25, high_frequency_cutoff = 2048) 
    
    return overlap_val

def ComputeOverlap(time1, strain1, time2, strain2):
    """ Compute overlap in one detector - Eq. 8 in arxiv.org/abs/2003.09456
    """
    inner_product = ComputeInnerProduct(time1, strain1, time2, strain2, normalized = True)
    return np.sqrt(inner_product)

def ComputeMultiDetectorInnerProduct(time1_H, strain1_H, time1_L, strain1_L, \
                                time2_H, strain2_H, time2_L, strain2_L):
    """ Given two strain waveforms in both H1 and L1, compute the multi-detector inner product.
        Use normalized = True for overlaps, and normalized = False for SNR.
        This is Eq. 3 in arxiv.org/abs/2003.09456
    """
    
    inner_H = ComputeInnerProduct(time1_H, strain1_H, time2_H, strain2_H, normalized = False)
    inner_L = ComputeInnerProduct(time1_L, strain1_L, time2_L, strain2_L, normalized = False)
    ## Sum the detector results
    return inner_H + inner_L

def ComputeMultiDetectorOverlap(time1_H, strain1_H, time1_L, strain1_L, \
                                time2_H, strain2_H, time2_L, strain2_L):
    
    """ Compute the overlap between two two-detector signals using Eq. 8 of arxiv.org/abs/2003.09456
    """
    
    ## numerator term
    O_AB = ComputeMultiDetectorInnerProduct(time1_H, strain1_H, time1_L, strain1_L, \
                                time2_H, strain2_H, time2_L, strain2_L)
    
    ## denominator terms
    O_AA = ComputeMultiDetectorInnerProduct(time1_H, strain1_H, time1_L, strain1_L, \
                                time1_H, strain1_H, time1_L, strain1_L)
    O_BB = ComputeMultiDetectorInnerProduct(time2_H, strain2_H, time2_L, strain2_L, \
                                time2_H, strain2_H, time2_L, strain2_L)
    
    result = O_AB / np.sqrt(O_AA * O_BB)
    return result
    

def ComputeSNR(time, strain):
    """ Given a time-domain strain and the corresponding time array, 
        compute the SNR using pycbc in one detector 
        Using Eq. 5 in arxiv.org/abs/2003.09456 """
    
    inner_product = ComputeInnerProduct(time, strain, time, strain, normalized = False)
    return np.sqrt(inner_product)


def ComputeMultiDetectorSNR(timeH, strainH, timeL, strainL):
    """ Compute an SNR in multiple detectors using Eq. 5 in arxiv.org/abs/2003.09456 
    """
    
    inner_product_H = ComputeInnerProduct(timeH, strainH, timeH, strainH, normalized = False)
    inner_product_L = ComputeInnerProduct(timeL, strainL, timeL, strainL, normalized = False)
    result = np.sqrt(inner_product_H + inner_product_L)
    return result